In [1]:
from asteroid.models import BaseModel
import soundfile as sf
import time
from pystoi import stoi
import os
import librosa
import matplotlib.pyplot as plt
import numpy as np

## 计算STOI

In [2]:
mixture_dir = '/home/oneran/Data/Libri2Mix/wav16k/both/dev/mix_both'
s1 = '/home/oneran/Data/Libri2Mix/wav16k/both/dev/s1'
s2 = '/home/oneran/Data/Libri2Mix/wav16k/both/dev/s2'
##################################################################
import torch
from asteroid.models import ConvTasNet
from asteroid.engine.system import System
from asteroid.engine.optimizers import make_optimizer
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
import threading
import time
# torch.set_default_dtype(torch.float16)
checkpoint = torch.load('/home/oneran/Downloads/16k_ep7.ckpt', map_location=torch.device('cpu'))
model = ConvTasNet(2, mask_act='relu', norm_type='gLN', kernel_size=32, stride=16)
model.sample_rate = 16000
model.eval()
temp = checkpoint['state_dict']
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")
optimizer = make_optimizer(model.parameters())
system = System(model, optimizer, loss_func, None)
system.load_state_dict(temp)
system.cpu()
to_save = system.model.serialize()
model.load_state_dict(to_save['state_dict'])
model.cuda()
model.sample_rate = 16000
##################################################################

histroys = []
print('load')

# def f(x, y):
#     if len(x) > len(y):
#         return x[:-(len(x) - len(y))]
#     else:
#         return np.pad(x, ((0, len(y.flatten()) - len(x.flatten()))), mode='constant', constant_values=0)
def read_audio(mixture, s1_path, s2_path, rec):
    # time_1 = time.time()
    wav1, _ = sf.read(s1_path)
    wav2, _ = sf.read(s2_path)
    wavmix, _ = sf.read(mixture)
    rec.append([wav1, wav2, wavmix])
    # time_2 = time.time()
    # print(time_2 - time_1)

def process(wav, rec):
    # time_1 = time.time()
    sep = model.separate(wav)
    rec.append(sep)
    # time_2 = time.time()
    # print(time_2 - time_1)

def cal_stoi(sep, wav1, wav2, histroys):
    sep1, sep2 = sep[0, 0], sep[0, 1]
    # sep1, sep2 = f(librosa.resample(sep1, 8000, 16000), wav1), f(librosa.resample(sep2, 8000, 16000), wav2)
    stoi_1_1, stoi_1_2 = stoi(wav1, sep1, 16000), stoi(wav2, sep2, 16000)
    stoi_2_1, stoi_2_2 = stoi(wav2, sep1, 16000), stoi(wav1, sep2, 16000)
    stoi_max_1, stoi_max_2 = max([stoi_1_1, stoi_1_2]), max([stoi_2_1, stoi_2_2])
    histroys.append([stoi_1_1, stoi_1_2] if stoi_max_1 > stoi_max_2 else [stoi_2_1, stoi_2_2]) 

audios = []
seps = []
stoi_threads_pool = []
for i, file in enumerate(os.listdir(mixture_dir)):
    mixture = mixture_dir + '/' + file
    s1_path = s1 + '/' + file
    s2_path = s2 + '/' + file
    #* 开始处理上一条
    if i != 0:
        t.join()
        audio = audios.pop()
        wavmix = audio[-1]
        wavmix = np.expand_dims(wavmix, axis=0).astype(np.float32)
        sep_t = threading.Thread(target=process, kwargs={'wav': wavmix, 'rec': seps})
        sep_t.start()

    #* 加载下一条

    t = threading.Thread(target=read_audio, kwargs={'mixture': mixture, 's1_path': s1_path, 's2_path': s2_path, 'rec': audios})
    t.start()

    if i != 0:
        sep_t.join()
        sep = seps.pop()
        wav1, wav2 = audio[0], audio[1]

    # wavmix = librosa.resample(wavmix, 16000, 8000)
    
    if i != 0:
        t_stoi = threading.Thread(target=cal_stoi, kwargs={'sep': sep, 'wav1': wav1, 'wav2': wav2, 'histroys': histroys})
        t_stoi.start()
        stoi_threads_pool.append(t_stoi)
        if len(stoi_threads_pool) >= 20:
            for _t in stoi_threads_pool:
                _t.join()
            stoi_threads_pool = []

        # historys.append([stoi_1_1, stoi_1_2])

/home/oneran/.local/lib/python3.10/site-packages/asteroid/models/base_models.py:55: UserWarning: Other sub-components of the model might have a `sample_rate` attribute, be sure to modify them for consistency.
  warnings.warn(


load


17